In [42]:
import csv
import os
import glob
import enchant
import pandas as pd
import itertools as it
from nltk.tokenize import sent_tokenize, word_tokenize,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


In [43]:

#Read the file and create a single tsv file for each record .  
with open('Airbnb_Texas_Rentals.csv') as csvin:
    reader = csv.reader(csvin)
    counter = 1
    for row in reader:
        with open("data/doc_"+str(counter)+".tsv",'w') as tsvout:
            writer = csv.writer(tsvout, delimiter='\t')
            writer.writerow(row)
            counter +=1

In [44]:
#Normalize the word. i.e. tokenize,stemming, and removing punctuations
def wordNorm(text):
    stemmer = SnowballStemmer("english")
    Normalized = []
    tokenizer = RegexpTokenizer(r'\w+')
    word_list = tokenizer.tokenize(text.lower())
    Remove_stopwords = [word for word in word_list if word not in stopwords.words('english')]
    stem_words = [stemmer.stem(word) for word in Remove_stopwords]
    d = enchant.Dict("en_US")
    for word in stem_words:
        if not word.isdigit() and len(word)>1 and d.check(word):#remove the punctuation
            Normalized.append(word)
    return Normalized

In [5]:
def readall_documents(path):
    words = []
    for file in glob.glob(path):
        df = pd.read_table(file,sep='\t',names=['index','average_rate_per_night','bedrooms_count',
                                                'city','date_of_listing','description','latitude',
                                                'longitude','title','url'])
        desc = wordNorm(str(list(df['description'])))
        title = wordNorm(str(list(df['title'])))
        for des, tit in it.zip_longest(desc,title):
            if(des is not None):
                words.append(des)
            if(tit is not None):
                words.append(tit)

    return words

In [15]:
#Read all the files inside test folder. Test folder contains all the tsv files
Allword = readall_documents('test/*')

In [45]:
def CreateVocabulary(allwords):
    # This function take in input all words, create the set and assign an ID to each term
    # vocabulary = {term : termID}
    Vocabulary = {}
    wordList = sorted(list(set(allwords)))
    for ID, element in enumerate(wordList):
        Vocabulary.update({element : ID})
    return Vocabulary

In [32]:
voc = CreateVocabulary(Allword)

In [39]:
def term_freq(term, text):
# Compute the term frequencies in a given text
    count = 0
    if len(text) <= 0:
        return 0
    else:
        for ter in text:
            if ter == term:
                count += 1
        return count / len(text)

In [46]:
#Build invertedIndex
def invertedIndex(vocabulary,path):
    # invertedIndex = {termID : (doc, TF)}
    invertedIndex = {}
    counter = 0
    for file in glob.glob(path):
        df = pd.read_table(file,sep='\t',names=['index','average_rate_per_night','bedrooms_count',
                                                'city','date_of_listing','description','latitude',
                                                'longitude','title','url'])
        
        desc = wordNorm(str(list(df['description'])))
        title = wordNorm(str(list(df['title'])))
        for word in vocabulary:
            tf = term_freq(word, desc+title)
            if(tf>0):
                invertedIndex[vocabulary[word]] = [(file, tf)]

    return invertedIndex

In [40]:
invIndex = invertedIndex(voc,'test/*')

dict